<a href="https://colab.research.google.com/github/sverdoot/VK_test_task/blob/master/toxic_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тестовое задание в лабораторию ВК, команда прикладных исследований. 
Выполнил Женя Лагутин

Построение легковестных моделей - одна из хайповых тем. Действительно, количество параметров SOTA моделей растёт с ужасающим темпом. Не очень ясно, как использовать в production такие language models, как BERT или, тем более, RoBERTa, которые при этом дают очень хорошее качество. 

В последнее время сжатию тяжеловесвтных моделей посвящают всё больше исследований: Distilling BERT into BiLSTM, DistilBERT от HuggingFace, BERT Quantization от Intel и, совсем недавно, Tiny BERT от Huawei.

Основная идея этих работ - взять хорошо обученную, но тяжёлую бертоподобную языковую модель и сжать ее, например, в бертоподобную модель меньшего размера. Среди способов это сделать:
* Сделать tensor-train разложение тензора эмбеддингов или feedforward слоёв трансформера
* Квантизовать эмбеддинги или feedforward слои трансформера
* Прунить BERT, уменьшая число слоёв attention, и число голов multi-head attention'ов

И наконец:
* Knowledge distillation


Я решил попробовать последний метод, так как он позволяет сжать BERT до произвольного размера, а так же не требует дополнительных манипуляций для ускорения модели, как, например, при квантизации. Моя teacher model - fine-tuned BERT-Base. В качестве student модели для дистилляции я выбрал обычную логистическую регрессию, принимающу на вход TF-IDF признаки. Такая модель будет, пожалуй, максимально лёгкой и быстрой.

Однако, обучить BERT на колабе оказалось не такой простой задачей, как я предполагал... Колаб вёл себя капризно как никогда. В результате особо хорошего качества от BERT'a получить не удалось и пришлось работать с тем, что есть. Результаты представлены ниже.

## Какие есть способы ускорения/уменьшения модели

**Ускорение:** "умная" квантизация (позволяющая превратить некоторые матричные операции в векторные; не любая реализация квантизации будет ускорять модель), прунинг (как уменьшение количества слоёв и голов)

**Уменьшение:** квантизация, матричные разложения (tensor-train, SVD в матричном случае), прунинг (в том числе уменьшение количества ненулевых весов), дистилляция

Если выбирать методом исключения, то останется дистилляция. 

Для квантизации нужно дополнительноучитряться, чтобы получить ускорение. 

Матричные разложения не дадут мне значительно уменьшить модель. Можно, конечно, использовать маленькую модель поверх разложенных эмбеддингов BERT'а, но всё-таки такой подход приводит к увеличению матричных умножений и, как следствие, замедлению инференса.

Прунинг помог бы мне уменьших модель в несколько раз, может в 10, но не в сотни раз.

Дистилляция же оставляет простор для выбора достаточно маленькой и достаточно быстрой модели.

## Что лучше - сразу маленькая модель или большая и потом её оптимизировать?

Существует ряд примеров, когда второй подход даёт лучший результат. В случае с дистилляцией это связано с тем, что логиты, на которых обучается student модель, содержат дополнительную информацию, которую маленькая модель попросту не может выучить сама по себе (lack of capacity). То есть то, что вероятности отсутствующих классов не нули, позволяет маленькой модели извлечь скрытые зависимости из данных.

## Недостатки моего модхода

Недостатки моего подхода становятся более явными в том случае, когда teacher модель обучена недостаточно хорошо, как в моём случае. Получается, что student модель обучается на недоостаточно правдивых данных. Я попытался это исправить, не позволяя модели учится на "ложных" логитах, подсовывая ей вместо этого реальный таргет.

Другой недостаток дистилляции как подхода: насколько я понимаю, пока нет гарантий, что дистилляция улучшает качество в общем случае.

## Код:

Далее, в качестве хранилища я использую папку 'toxic_distill' на гугл диске. Чтобы всё работало - создайте такую и у себя)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Код для загруки данных с kaggle, используя kaggle api

In [0]:
#from google.colab import files
#files.upload()

#!mkdir -p ~/.kaggle
#!cp "kaggle.json" ~/.kaggle/

#!pip install -q kaggle
##!chmod 600 /root/.kaggle/kaggle.json
#!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

#import zipfile
#with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
#    zip_ref.extractall('drive/My Drive/toxic_distill')
#with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
#    zip_ref.extractall('drive/My Drive/toxic_distill')

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('drive/My Drive/toxic_distill/train.csv')
#test_df = pd.read_csv('test.csv')

In [5]:
df[:5]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
label_names = list(df.columns[-6:])
n_labels = 6

Я разделил данные на train и dev в соотношении 4:1

In [0]:
train_df = df[:int(len(df)*0.8)]
dev_df = df[-int(len(df)*0.2):]

Как видно из следующей клетки, объектов каждого класса довольно мало по отношению к общему колиству комментариев, так что, построить уверенный классификатор довольно непросто.

In [8]:
print('Train:\t', [f'{x[0]}: {x[1]*100.:.2f}%' for x in zip(label_names, np.array(train_df.iloc[:, 3:]).sum(0) / len(train_df))])
print('Dev:\t', [f'{x[0]}: {x[1]*100.:.2f}%' for x in zip(label_names, np.array(dev_df.iloc[:, 3:]).sum(0) / len(dev_df))])

Train:	 ['toxic: 1.01%', 'severe_toxic: 5.31%', 'obscene: 0.30%', 'threat: 4.93%', 'insult: 0.86%']
Dev:	 ['toxic: 0.97%', 'severe_toxic: 5.23%', 'obscene: 0.29%', 'threat: 4.96%', 'insult: 0.96%']


Pretrained bert я позаимствую из репозитория hugging face - pytorch-transformers.

In [0]:
!pip install pytorch-transformers==1.2.0

In [0]:
import torch
from torch import nn
from torch import functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import *
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np
import time
from torch import optim
from torch.optim.lr_scheduler import LambdaLR

In [0]:
sigmoid = lambda x : 1 / (1 + np.exp(-x)) # sigmoid for numpy arrays

if torch.cuda.is_available():
  device = 0
else:
  device = 'cpu'

## Teacher:

In [0]:
teacher_n_epoch = 4
teacher_lr = 3e-5
teacher_warmup_proportion = 0.1
teacher_batch_size = 32
teacher_max_seq_len = 150
teacher_dropout = 0.25
max_seq_len = 128

teacher_path = None # teacher model storage
logits_path = None # teacher's logits storage

In [13]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 939624.88B/s]


In [0]:
PAD_TOKEN = bert_tokenizer.pad_token
PAD_TOKEN_ID = bert_tokenizer.pad_token_id
CLS_TOKEN = bert_tokenizer.cls_token

bert_config = BertConfig()
bert_hidden_size = bert_config.hidden_size

Здесь всё просто - я не стал замарачиваться и просто добавил линейный слой поверх выхода берта cls токена. 

In [0]:
class BertTeacher(BertModel):
    def __init__(self, bert_config, num_labels=6, pad_id=0, dropout=0.):
        super(BertTeacher, self).__init__(bert_config)
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(bert_config.hidden_size, num_labels)
        self.pad_id = pad_id

    def forward(self, input_ids):
        _, pooled_output = self.bert(input_ids, (input_ids!=0).long())
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

In [0]:
class TeacherDataset(Dataset):

    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        comment_text = self.dataframe.iloc[idx, 1]
        labels = torch.LongTensor((self.dataframe.iloc[idx, 2:]))
        
        if self.transform:
          comment_text = torch.LongTensor(self.transform(comment_text))

        sample = {'comment_text': comment_text, 'labels': labels}

        return sample

Следующая функция позволяет собрать статистику длин комментариев в токенах. Как оказалось, чтобы покрыть 85% комментариев целиком, длина должна быть порядка 150 токенов (!) - очередной вызов для колаба.

In [0]:
def seq_len_stat(texts, tokenizer, quantile=0.85):
  lens = []
  for comment in train_df['comment_text']:
    lens.append(len(bert_tokenizer.tokenize(comment)))
  print(f'Mean length: {np.mean(lens)}, median: {np.median(lens)}, length covering {quantile:.2f} sequences: {np.quantile(lens, quantile)}')

In [0]:
class TextProcessing():
  
  def __init__(self, tokenizer, max_len=100):
    self.tokenizer = tokenizer
    self.pad_token = tokenizer.pad_token
    self.cls_token = tokenizer.cls_token
    self.max_len = max_len
    
  def __call__(self, text):
    tokens = self.tokenizer.tokenize(text)
    tokens = tokens[:self.max_len]
    tokens += [self.pad_token]*(self.max_len - len(tokens))
    tokens = [self.cls_token] + tokens
    return torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)).long()

In [0]:
teacher_transform = TextProcessing(bert_tokenizer, max_len=max_seq_len)

In [0]:
train_dataset = TeacherDataset(train_df, teacher_transform)
dev_dataset = TeacherDataset(dev_df, teacher_transform)

In [0]:
train_batcher = DataLoader(train_dataset, batch_size=teacher_batch_size, shuffle=False)
dev_batcher = DataLoader(dev_dataset, batch_size=teacher_batch_size, shuffle=False)

In [0]:
# function for printing classification report and roc_auc_score

def report(y_true, probs, label_names):
  y_pred = (probs >= 0.5)
  avg_roc_auc = 0
  for i, label_name in enumerate(label_names):
    print(f'CLASS: {label_name.upper()}')
    print(classification_report(y_true[:,i], y_pred[:,i]))
    roc_auc = roc_auc_score(y_true[:,i], probs[:,i])
    print(f'ROC-AUC score: {roc_auc}')
    avg_roc_auc += roc_auc
  avg_roc_auc /= len(label_names)
  print(avg_roc_auc)
  
  return avg_roc_auc

In [0]:
criterion = nn.BCEWithLogitsLoss() # for multilabel classification

In [24]:
teacher_model = BertTeacher(bert_config, n_labels, PAD_TOKEN_ID, teacher_dropout).to(device)

teacher_optimizer = AdamW(teacher_model.parameters(), lr=teacher_lr)

100%|██████████| 440473133/440473133 [00:16<00:00, 26996330.95B/s]


Следуя заветам fine-tuning'a BERT'a, я добавил warmup, однако пришлось от него отказаться, так как процесс обучения на колабе не позволяет планировать наперёд)

In [0]:
# code from hugging_face/transformers

class WarmupLinearSchedule(LambdaLR):
    """ Linear warmup and then linear decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Linearly decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps.
    """
    def __init__(self, optimizer, warmup_steps, t_total, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        super(WarmupLinearSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        return max(0.0, float(self.t_total - step) / float(max(1.0, self.t_total - self.warmup_steps)))

In [0]:
total_n_steps = len(train_df) / teacher_batch_size * teacher_n_epoch
warmup_steps = int(total_n_steps * teacher_warmup_proportion)
teacher_scheduler = WarmupLinearSchedule(teacher_optimizer, warmup_steps, total_n_steps)

In [0]:
#teacher_path = 'drive/My Drive/toxic_distill/teacher.pt'

if teacher_path is not None:
  chkpnt = torch.load(teacher_path)
  teacher_model.load_state_dict(chkpnt['model_state_dict'])
  teacher_optimizer.load_state_dict(chkpnt['optimizer'])
  lock_loss = chkpnt['loss']
else:
  lock_loss = float('Inf')

*Комментарий: в процеесе обучения я вывожу следующие метрики: процент объектов, для которых предсказание целиком верное (full_match_acc), и общие процент верно предсказанных меток (part-match-acc)*

In [0]:
# training loop

print_every = 300 

for epoch in range(teacher_n_epoch):
    teacher_model.train()
    train_loss = 0
    train_logits =[]
    train_part_match = 0
    train_exact_match = 0
    n_ex = 0  # number of processed examples
    epoch_time = time.time()
    
    for batch_id, batch in enumerate(train_batcher):
      teacher_optimizer.zero_grad()
      comment = batch['comment_text'].to(device)
      target = batch['labels'].to(device)
      n_ex += comment.size(0)

      out = teacher_model(comment)
      train_logits.append(out.detach().cpu().numpy())
      pred = (out.sigmoid() >= 0.5).long()
      
      loss = criterion(out.view(-1, n_labels), target.float().view(-1, n_labels))
      train_loss += loss.item()
      loss.backward()
      teacher_optimizer.step()
      #teacher_scheduler.step()
      
      train_exact_match += torch.sum(torch.sum(pred == target, 1) == n_labels).item()
      train_part_match += torch.sum(pred == target).item()

      if batch_id % print_every == 0 and batch_id > 0:
        print(f'\tIteration {batch_id}. Training loss: {(train_loss / batch_id):.7f}, full-match acc: {(train_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(train_part_match / n_ex / n_labels):.3f}')
    
    print(f'Epoch {epoch}. Training loss: {(train_loss / batch_id):.7f}, full-match acc: {(train_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(train_part_match / n_ex / n_labels):.3f}')
    train_logits = np.vstack(train_logits)
    
    with torch.no_grad():
      teacher_model.eval()
      val_loss = 0
      n_ex = 0
      val_exact_match = 0
      val_part_match = 0
      y_pred = []
      y_true = []
      dev_logits = []
      
      for batch_id, batch in enumerate(dev_batcher):
        comment = batch['comment_text'].to(device)
        target = batch['labels'].to(device)
        n_ex += comment.size(0)
        
        out = teacher_model(comment)
        dev_logits.append(out.detach().cpu().numpy())
        pred = (out.sigmoid() >= 0.5).long()
        
        loss = criterion(out.view(-1, n_labels), target.float().view(-1, n_labels))
        val_loss += loss.item()
        
        val_exact_match += torch.sum(torch.sum(pred == target, 1) == n_labels).item()
        val_part_match += torch.sum(pred == target).item()
        
        y_pred += pred.tolist()
        y_true += target.tolist()

        if batch_id % print_every == 0 and batch_id > 0:
          print(f'\tIteration {batch_id}. Validation loss: {(val_loss / batch_id):.7f}')
      
      print(f'Epoch {epoch}. Validation loss: {(val_loss / batch_id):.7f}, full-match acc: {(val_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(val_part_match / n_ex / n_labels):.3f}')
      dev_logits = np.vstack(dev_logits)
      
      y_pred = np.array(y_pred)
      y_true = np.array(y_true)
      avg_roc_auc = report(y_true, sigmoid(dev_logits), label_names)
   
      if val_loss < lock_loss:
        lock_loss = val_loss
        lock_avg_roc_auc = avg_roc_auc
        torch.save({'model_state_dict': teacher_model.state_dict(),
                     'optimizer': teacher_optimizer.state_dict(),
                     'loss': val_loss / batch_id,
                     'full-match acc': val_exact_match / n_ex,
                     'avg_roc_auc': avg_roc_auc,
                     'part-match acc': val_part_match / n_ex / n_labels,
                     'pred_true': (y_pred, y_true),
                     'epoch': epoch}, 'drive/My Drive/toxic_distill/teacher.pt')
        torch.save({'train_logits':train_logits, 'dev_logits':dev_logits}, 'drive/My Drive/toxic_distill/logits.pt')
     

In [0]:
logits = 'drive/My Drive/toxic_distill/logits.pt'
logits = torch.load(logits)
dev_logits = logits['dev_logits']
train_logits = logits['train_logits']

Далее - результаты fine-tuning'a BERT Base, качество на валидационном сете. Not great, not terrible.

In [0]:
# shows the report
report(np.array(dev_df.iloc[:, 2:]), sigmoid(dev_logits), label_names)

### Report:

CLASS: TOXIC

              precision    recall  f1-score   support

           0       0.98      0.98      0.98     28877
           1       0.80      0.82      0.81      3037

ROC-AUC score: 0.9800327023719385

CLASS: SEVERE_TOXIC

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31603
           1       0.65      0.15      0.25       311

ROC-AUC score: 0.9890117884327193

CLASS: OBSCENE

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     30245
           1       0.84      0.81      0.83      1669

ROC-AUC score: 0.9889399046988044

CLASS: THREAT

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31822
           1       0.56      0.32      0.40        92

ROC-AUC score: 0.9725331873218693

CLASS: INSULT

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     30332
           1       0.77      0.73      0.75      1582

ROC-AUC score: 0.9855226683947542

CLASS: IDENTITY_HATE

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31609
           1       0.75      0.34      0.47       305

ROC-AUC score: 0.9858376608861659

**AVG ROC-AUC: 0.9836463186843751**


In [0]:
# supplementary stuff

def collect_logits(model, batcher, device):
  with torch.no_grad():
    model.eval()
    logits = []
    
    for batch_id, batch in enumerate(batcher):
      comment = batch['comment_text'].to(device)
      target = batch['labels'].to(device)
      
      out = model(comment)
      logits.append(out.detach().cpu().numpy())
  logits = np.vstack(logits)
  
  return logits

## Student:

In [0]:
student_batch_size = 64
student_lr = 0.1
student_momentum = 0.9
n_word_features = 10000
n_char_features = 10000
student_n_epoch = 6

In [0]:
class MultilabelLinearRegression(nn.Module):
  
  def __init__(self, input_dim, n_classes):
    super(MultilabelLinearRegression, self).__init__()
    
    self.input_dim = input_dim
    self.n_classes = n_classes
    self.lin = nn.Linear(input_dim, n_classes)
    self.sigmoid = nn.Sigmoid()
    
  def forward(self, inp):
    output = self.lin(inp.float())
    return output

Как было сказано выше, в качестве признаков для логистической регрессии я выбрал tf-idf. Код для сбора tf-idf я загуглил.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

In [35]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=n_word_features)
word_vectorizer.fit(train_df['comment_text'])

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=n_char_features)
char_vectorizer.fit(train_df['comment_text'])

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(2, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents='unicode',
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
class StudentDataset(Dataset):

    def __init__(self, dataframe, logits, transform=None):
        assert len(dataframe) == len(logits)
        self.dataframe = dataframe
        self.transform = transform
        self.logits = logits

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        logits = torch.FloatTensor(self.logits[idx])
        comment_text = self.dataframe.iloc[idx, 1]
        labels = torch.LongTensor((self.dataframe.iloc[idx, 2:]))
        
        if self.transform:
          comment_text = self.transform(comment_text)

        sample = {'comment_text': comment_text, 'labels': labels, 'logits': logits}

        return sample

In [0]:
from scipy.sparse import hstack

def lr_transform(text):  
  return torch.tensor(hstack([word_vectorizer.transform([text]), char_vectorizer.transform([text])]).todense()).float()

In [0]:
lr_train_dataset = StudentDataset(train_df, train_logits, lr_transform)
lr_dev_dataset = StudentDataset(dev_df, dev_logits, lr_transform)

In [0]:
lr_train_batcher = DataLoader(lr_train_dataset, batch_size=student_batch_size, shuffle=False)
lr_dev_batcher = DataLoader(lr_dev_dataset, batch_size=student_batch_size, shuffle=False)

In [0]:
student_model = MultilabelLinearRegression(n_char_features+n_word_features, n_labels).to(device)

student_optimizer = optim.SGD(student_model.parameters(), lr=student_lr, nesterov=True, momentum=student_momentum)

Здесь развилка: как обучать логистическую регрессию на логитах? 

* Первый способ - минимизация $l_p$ расстояния между логитами учителя и студента. 
* Второй же - более изощрённый - минимизация KL-дивергенции между вероятностями одной модели и другой. В этом случае, моя цель, сделать, чтобы апостериорное распределение студента описывало апостериорное распределение учителя. Кроме того, как было сказано выше, в случаях, когда учитель ошибался, я подсовывал студенту правильный овет в качестве таргета, что соответствует минимизации классической бинарной кросс энтропии.

Проведя эксперимент, я пришёл к выводу, что первый способ ($p=2$, т.е. обычный метод наименьших квадратов) даёт лучший результат. Возможно потому, что непосредственно логиты оказались более полезными для обучения линейной модели, чем нормированные вероятности.


In [0]:
distill_criterion = nn.MSELoss() #nn.KLDivLoss()

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
# training loop

lock_loss = 0
print_every = 100

for epoch in range(student_n_epoch):
    student_model.train()
    train_loss = 0
    train_part_match = 0
    train_exact_match = 0
    n_ex = 0  # number of processed examples
    
    for batch_id, batch in enumerate(lr_train_batcher):
      student_optimizer.zero_grad()
      comment = batch['comment_text'].to(device).squeeze(1)
      logits = batch['logits'].to(device)
      labels = batch['labels'].to(device)
      n_ex += comment.size(0)

      out = student_model(comment)
      pred = (out.sigmoid() >= 0.5).long()
      
      # in case of minimizing for KLDivLoss:
      
      #out_bin = torch.stack([out, 1-out], -1)
      #probs = logits.sigmoid()
      #print(probs[torch.sum((probs>=0.5).long() != labels,-1).type(torch.BoolTensor)], labels[torch.sum((probs>=0.5).long() != labels,-1).type(torch.BoolTensor)])
      #probs[torch.sum((probs>=0.5).long() != labels,-1).type(torch.BoolTensor)] = labels[torch.sum((probs>=0.5).long() != labels,-1).type(torch.BoolTensor)].float()
      #probs_bin = torch.stack([probs, 1-probs], -1)
      #loss = distill_criterion(out_bin.view(-1,2).log(), probs_bin.view(-1,2))
      
      loss = distill_criterion(out, logits)
      
      train_loss += loss.item()
      loss.backward()
      student_optimizer.step()
      
      train_exact_match += torch.sum(torch.sum(pred == labels, 1) == n_labels).item()
      train_part_match += torch.sum(pred == labels).item()

      if batch_id % print_every == 0 and batch_id > 0:
        print(f'\tIteration {batch_id}. Training loss: {(train_loss / batch_id):.7f}, full-match acc: {(train_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(train_part_match / n_ex / n_labels):.3f}')
    
    print(f'Epoch {epoch}. Training loss: {(train_loss / batch_id):.7f}, full-match acc: {(train_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(train_part_match / n_ex / n_labels):.3f}')
    
    with torch.no_grad():
      teacher_model.eval()
      val_loss = 0
      n_ex = 0
      val_exact_match = 0
      val_part_match = 0
      y_pred = []
      y_true = []
      student_dev_probs = []
      
      for batch_id, batch in enumerate(lr_dev_batcher):
        comment = batch['comment_text'].to(device).squeeze(1)
        logits = batch['logits'].to(device)
        labels = batch['labels'].to(device)
        n_ex += comment.size(0)
        
        out = student_model(comment.float())
        student_dev_probs.append(out.sigmoid().detach().cpu().numpy())
        pred = (out.sigmoid() >= 0.5).long()
        
        loss = distill_criterion(out, logits)
        
        val_loss += loss.item()
        val_loss_list.append(loss.item())
        
        val_exact_match += torch.sum(torch.sum(pred == labels, 1) == n_labels).item()
        val_part_match += torch.sum(pred == labels).item()
        
        y_pred += pred.tolist()
        y_true += labels.tolist()

        if batch_id % print_every == 0 and batch_id > 0:
          print(f'\tIteration {batch_id}. Validation loss: {(val_loss / batch_id):.7f}')
      
      val_loss /= batch_id
      print(f'Epoch {epoch}. Validation loss: {val_loss:.7f}, full-match acc: {(val_exact_match / n_ex):.3f}, ' + \
        f'part-match acc {(val_part_match / n_ex / n_labels):.3f}')
      
      y_pred = np.array(y_pred)
      y_true = np.array(y_true)
      student_dev_probs = np.vstack(student_dev_probs)
      roc_auc = report(y_true, student_dev_probs, label_names)
      
      if val_loss < lock_loss or epoch == 0:
        lock_loss = val_loss
        torch.save({'model_state_dict': student_model.state_dict(),
                     'optimizer': student_optimizer.state_dict(),
                     'loss': val_loss,
                     'full-match acc': val_exact_match / n_ex,
                     'part-match acc': val_part_match / n_ex / n_labels,
                     'pred_true': (y_pred, y_true),
                     'epoch': epoch}, 'drive/My Drive/toxic_distill/student.pt')

### Report:

CLASS: TOXIC

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     28877
           1       0.96      0.52      0.68      3037


ROC-AUC score: 0.9768482353862907

CLASS: SEVERE_TOXIC

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31603
           1       0.52      0.30      0.38       311

ROC-AUC score: 0.9894976188206317

CLASS: OBSCENE

              precision    recall  f1-score   support

           0       0.97      1.00      0.99     30245
           1       0.92      0.52      0.66      1669

ROC-AUC score: 0.9881755557098555

CLASS: THREAT

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     31822
           1       0.67      0.09      0.15        92

ROC-AUC score: 0.9733107803461101

CLASS: INSULT

              precision    recall  f1-score   support

           0       0.97      1.00      0.98     30332
           1       0.88      0.41      0.56      1582

ROC-AUC score: 0.9825044892986223

CLASS: IDENTITY_HATE

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     31609
           1       0.61      0.12      0.20       305

ROC-AUC score: 0.9792807506058919

**AVG ROC-AUC: 0.9816029050279004**

In [44]:
student_model.load_state_dict(torch.load('drive/My Drive/toxic_distill/student.pt')['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [45]:
!pip install torchsummary

In [46]:
from torchsummary import summary

summary(student_model, (1, (n_word_features+n_char_features)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 6]         120,006
Total params: 120,006
Trainable params: 120,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.08
Forward/backward pass size (MB): 0.00
Params size (MB): 0.46
Estimated Total Size (MB): 0.53
----------------------------------------------------------------


Вес student модели, кажется, достаточно мал $\sim 0.53 ~Mb$. Плюс спарс-матрица tf-idf. 

В скорости инференса у такой модели тоже не занимать - одно умножение матрицы на вектор + сигмоида.

Куда уж меньше?

## Pure student:

Теперь проверим, сколько даёт логистическаая регрессия без дистилляции:

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

train_word_features = word_vectorizer.transform(train_df['comment_text'])
train_char_features = char_vectorizer.transform(train_df['comment_text'])
dev_word_features = word_vectorizer.transform(dev_df['comment_text'])
dev_char_features = char_vectorizer.transform(dev_df['comment_text'])

train_features = hstack([train_char_features, train_word_features])
dev_features = hstack([dev_char_features, dev_word_features])

scores = []
for i, name in enumerate(label_names):
    lr_clf = LogisticRegression(C=0.1, solver='sag')
    lr_clf.fit(train_features, train_df[name])
    probs = lr_clf.predict_proba(dev_features)
    scores.append(roc_auc_score(dev_df[name], probs[:,1]))
    print(f'roc-auc for class {name}: {scores[-1]}')
print(f'avg roc-auc: {np.mean(scores)}')

roc-auc for class toxic: 0.9680880663229708
roc-auc for class severe_toxic: 0.9889524713403312
roc-auc for class obscene: 0.9833513623165955
roc-auc for class threat: 0.9782806808524592
roc-auc for class insult: 0.9746717031059396
roc-auc for class identity_hate: 0.9777377163279395
avg roc-auc: 0.9785136667110393


В итоге мой подход сократил количество ошибок на $14\%$. Сократил бы на $46$ - выиграл бы challenge.

## Интерпретируемость:

В качестве интерпретации модели, я вывел 10 самых значимых признаков для каждого класса. Значимость оценивается как величина абсолютного значения нейрона между признаком и классом. 

Получилось забавно, но немного однообразно. Уж простите за обсценную лексику, но больше её избегать не получится.

In [0]:
weights = student_model.lin.weight.data
abs_weights = weights.abs()

In [0]:
k = 10
topk = torch.topk(abs_weights, k, dim=1)

In [0]:
word_features = word_vectorizer.get_feature_names()
char_features = char_vectorizer.get_feature_names()

In [0]:
for i, name in enumerate(label_names):
  class_features = [((word_features+char_features)[idx.item()]) for idx, val in zip(topk.indices[i], topk.values[i])]
  print(name)
  print(class_features)

toxic
['idiot', 'stupid', 'crap', 'moron', 'uck', 'penis', 'ass', 'asshole', 'gay', 'hell']
severe_toxic
['stupid', 'ass', 'idiot', 'asshole', 'uck', 'penis', 'fuc', 'cunt', 'fuck', 'gay']
obscene
['stupid', 'idiot', 'ass', 'crap', 'uck', 'penis', 'bullshit', 'asshole', 'cunt', 'damn']
threat
['die', 'kill', 'gay', 'stupid', 'nazi', 'hell', 'crap', 'penis', 'hate', 'ass']
insult
['idiot', 'stupid', 'moron', 'crap', 'gay', 'asshole', 'ass', 'uck', 'idiots', 'cunt']
identity_hate
['gay', 'stupid', 'nazi', 'nigger', 'idiot', 'fag', 'faggot', 'jew', 'nigg', 'racist']


## Как можно улучшить модель?

1. Лучше обучить учителя.
2. Взять BERT Large  в качестве учителя)
3. Сделать ансамбль студентов, можно даже с разными архитектурами.
4. Увеличить количество признаков. Можно собрать другие полезные статистики, кроме tf-idf на униграммах и символах. Например, на биграммах и триграммах), чтобы учитывать наличие специфических словосочетаний.
